In [1]:
import pandas as pd
# from json_functions import create_df_fixed, create_example

# players_list, df = create_df_fixed('very-big.json')

In [22]:
from json_functions import make_player_list, create_example

def create_df_fixed(file_to_use, mirror=True):
    import numpy as np
    raw_dataset = pd.read_json(file_to_use)
    processed_data = []

    players_list = make_player_list(file_to_use)
    numPlayers = len(players_list)

    for index, game in raw_dataset.iterrows():

        if len(game.team1) > 5 or len(game.team2) > 5:
            continue
        roster_team1 = [0] * numPlayers
        roster_team2 = [0] * numPlayers
        rating_vector = [[0, 0.]] * 10
        rating_vector = np.array(rating_vector)

        # populate the team 1 roster and performance vectors
        for i, player in enumerate(game.team1):
            player_index = players_list.index(player['name'])
            roster_team1[player_index] = 1
            rating_vector[i][0] = player_index
            rating_vector[i][1] = round(float(player['rating']), 3)
        # populate the team 2 roster and performance vectors
        for i, player in enumerate(game.team2):
            player_index = players_list.index(player['name'])
            roster_team2[player_index] = 1
            rating_vector[i+5][0] = player_index
            rating_vector[i+5][1] = round(float(player['rating']), 3)
        
        # sort ratings acording to corresponding player index
        rating_vector = rating_vector[rating_vector[:,0].argsort()]

        # put it all together in row, keep only ratings part of rating vector
        row = [roster_team1+roster_team2, rating_vector[:,1].tolist()]
        processed_data.append(row)

        if mirror:
            # now flip the teams around and add that to the training data
            temp = rating_vector[:,1].tolist()
            rating_vector_mirrored = temp[5:10] + temp[0:5]
            
            row_mirrored = [roster_team2+roster_team1, rating_vector_mirrored]
            processed_data.append(row_mirrored)

    return players_list, pd.DataFrame(processed_data, columns=['rosters vector',
                                                               'rating vector'])

In [7]:
players_list, df = create_df_fixed('../data/very-big.json')

df

,rosters vector,rating vector
0,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.55, 1.47, 1.07, 0.88, 0.86, 1.32, 1.11, 1.0..."
1,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1.32, 1.11, 1.06, 0.82, 0.68, 1.55, 1.47, 1.0..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1.75, 1.49, 1.32, 1.19, 0.99, 0.9, 0.88, 0.83..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.9, 0.88, 0.83, 0.79, 0.56, 1.75, 1.49, 1.32..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.32, 0.75, 0.96, 0.83, 1.03, 1.48, 1.28, 1.2..."
...,...,...
17483,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.34, 0.46, 0.71, 0.74, 2.09, 1.29, 0.81, 0.5..."
17484,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.86, 0.65, 1.15, 0.85, 0.77, 1.48, 1.14, 1.1..."
17485,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.48, 1.14, 1.16, 0.94, 1.04, 0.86, 0.65, 1.1..."
17486,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.03, 1.11, 1.05, 0.94, 1.14, 0.8, 1.09, 1.42..."


In [8]:
import numpy as np
import tensorflow as tf

In [9]:
features, outputs = df["rosters vector"], df['rating vector']

features = pd.DataFrame(features.values.tolist(), index= df.index)
outputs = pd.DataFrame(outputs.values.tolist(), index= df.index)

print(np.shape(features))
print(np.shape(outputs))


(17488, 1068)
(17488, 10)


In [10]:
import tensorflow as tf

In [11]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [12]:
tf.config.list_physical_devices('GPU')

[]

In [13]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(2000, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(2000, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))	
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [14]:
n_inputs, n_outputs = features.shape[1], outputs.shape[1]

In [15]:
model = get_model(n_inputs, n_outputs)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.33, random_state=42)

In [17]:
model.fit(X_train, y_train, verbose=1, epochs=20, batch_size=2048)
print("Evaluation on test set:")
mae = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/20
6/6 [==============================] - 2s 348ms/step - loss: 0.6808
Epoch 2/20
6/6 [==============================] - 2s 274ms/step - loss: 0.4917
Epoch 3/20
6/6 [==============================] - 2s 282ms/step - loss: 0.4425
Epoch 4/20
6/6 [==============================] - 2s 280ms/step - loss: 0.4022
Epoch 5/20
6/6 [==============================] - 2s 289ms/step - loss: 0.3704
Epoch 6/20
6/6 [==============================] - 2s 301ms/step - loss: 0.3451
Epoch 7/20
6/6 [==============================] - 2s 287ms/step - loss: 0.3228
Epoch 8/20
6/6 [==============================] - 2s 277ms/step - loss: 0.3008
Epoch 9/20
6/6 [==============================] - 2s 275ms/step - loss: 0.2796
Epoch 10/20
6/6 [==============================] - 2s 271ms/step - loss: 0.2613
Epoch 11/20
6/6 [==============================] - 2s 290ms/step - loss: 0.2492
Epoch 12/20
6/6 [==============================] - 2s 307ms/step - loss: 0.2429
Epoch 13/20
6/6 [==============================] 

In [23]:
example = create_example(['EliGE', 'Stewie2K', 'Grim', 'FalleN', 'NAF'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
model.predict(np.asarray([example]))

array([[1.1291966 , 1.0445014 , 1.0598189 , 0.96564984, 0.9988414 ,
        1.0738106 , 1.0584363 , 1.0242057 , 1.083325  , 1.0856991 ]],
      dtype=float32)

In [24]:
def print_prediction(example, players_list, model):
    player_indices = [i for i, x in enumerate(example) if x == 1]
    example_formatted = np.asarray([example])
    pred = model.predict(example_formatted)

    print("Team 1:")
    for i, player  in enumerate(player_indices):
        if player >= len(players_list): player =player - len(players_list)
        print(players_list[player]+': ', round(pred[0][i], 2))
        if i == 4:
            print("Agregate Rating:", round(np.sum(pred[0][0:5]), 2))
            print("Team 2:")
        if i == 9:
            print("Agregate Rating:", round(np.sum(pred[0][5:10]), 2))


In [25]:
print_prediction(example, players_list, model)

Team 1:
EliGE:  1.13
NAF:  1.04
Stewie2K:  1.06
FalleN:  0.97
Grim:  1.0
Agregate Rating: 5.2
Team 2:
device:  1.07
dupreeh:  1.06
Xyp9x:  1.02
gla1ve:  1.08
Magisk:  1.09
Agregate Rating: 5.33


Compare these results to a couple real matches played between these two teams on March 26th that the model hasn't seen:

https://www.hltv.org/stats/matches/mapstatsid/117239/liquid-vs-astralis?rankingFilter=Top10

https://www.hltv.org/stats/matches/mapstatsid/117231/astralis-vs-liquid?rankingFilter=Top10

In [26]:
print(players_list.index('EliGE'), 
      players_list.index('NAF'), 
      players_list.index('Stewie2K'), 
      players_list.index('FalleN'), 
      players_list.index('Grim'), )
print(players_list.index('device'), 
      players_list.index('dupreeh'), 
      players_list.index('Xyp9x'), 
      players_list.index('gla1ve'), 
      players_list.index('Magisk'), )

32 53 87 108 474
111 112 113 115 183


Here's a hypothetical game that could never happen: 2016 Astralis vs current Astralis. Current Astralis is the most dominant CSGO roster of all time, so we would expect to see higher ratings accross the board for them, which we do. (They are team 2 here).

In [27]:
example2 = create_example(['device', 'Xyp9x', 'Kjaerbye', 'dupreeh', 'karrigan'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
print_prediction(example2, players_list, model)

Team 1:
karrigan:  1.04
device:  1.0
dupreeh:  1.15
Xyp9x:  0.99
Kjaerbye:  1.08
Agregate Rating: 5.26
Team 2:
device:  1.04
dupreeh:  0.94
Xyp9x:  0.94
gla1ve:  0.99
Magisk:  1.14
Agregate Rating: 5.04


Lets try it with the teams switched now

In [28]:
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['device', 'Xyp9x', 'Kjaerbye', 'dupreeh', 'karrigan'], players_list)
print_prediction(example2, players_list, model)

Team 1:
device:  1.01
dupreeh:  1.1
Xyp9x:  1.03
gla1ve:  1.0
Magisk:  1.06
Agregate Rating: 5.2
Team 2:
karrigan:  1.05
device:  1.05
dupreeh:  1.01
Xyp9x:  1.03
Kjaerbye:  1.09
Agregate Rating: 5.24


In [29]:
# One of the most unbalanced possible matchups, "all-stars" vs low rated players
example2 = create_example(['device', 's1mple', 'NiKo', 'ZywOo', 'electronic'], ['daps', 'karrigan', 'Golden', 'TACO', 'AdreN'], players_list)
print_prediction(example2, players_list, model)

Team 1:
s1mple:  0.99
electronic:  1.04
NiKo:  0.99
device:  0.93
ZywOo:  0.97
Agregate Rating: 4.92
Team 2:
karrigan:  0.98
Golden:  1.01
AdreN:  0.99
TACO:  0.9
daps:  0.97
Agregate Rating: 4.85


In [30]:
# Astralis vs a random team of players that generally get low rating
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['daps', 'karrigan', 'Golden', 'TACO', 'AdreN'], players_list)
print_prediction(example2, players_list, model)

Team 1:
device:  0.93
dupreeh:  1.09
Xyp9x:  1.0
gla1ve:  1.0
Magisk:  1.06
Agregate Rating: 5.07
Team 2:
karrigan:  1.02
Golden:  1.01
AdreN:  0.97
TACO:  0.98
daps:  1.07
Agregate Rating: 5.04


In [31]:
# Astralis vs All-Stars
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['device', 's1mple', 'NiKo', 'ZywOo', 'electronic'], players_list)
print_prediction(example2, players_list, model)

Team 1:
device:  1.12
dupreeh:  0.97
Xyp9x:  1.14
gla1ve:  0.96
Magisk:  0.95
Agregate Rating: 5.15
Team 2:
s1mple:  1.07
electronic:  0.98
NiKo:  0.98
device:  1.0
ZywOo:  1.09
Agregate Rating: 5.12


In [32]:
model.save('../models/small_output.h5')